### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
%matplotlib inline

In [ ]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-07-14 11:25:27--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-07-14 11:25:28--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf5aa4d61098ce88e54ff915de7.dl.dropboxusercontent.com/cd/0/inline/Ctc4_jpm9vYiO8f43_Zd-ahFy0yUODiK1hJs__ifs3TrjIzRLkdcpeSHwSSRwoPvttv0Up4OkzEHYu7S1lDMAcdTne35ifzHgPp3fb_wC8vxafdETIK_h-VsLLH2UAo_aX4/file?dl=1# [following]
--2025-07-14 11:25:28--  https://ucf5aa4d61098ce88e54ff915de7.dl.dropboxuse

,author,day,id,link,month,summary,tag,title,year
27246,"[{'name': 'Gong Cheng'}, {'name': 'Junwei Han'...",1,1703.00121v1,"[{'rel': 'related', 'href': 'http://dx.doi.org...",3,Remote sensing image scene classification play...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Remote Sensing Image Scene Classification: Ben...,2017
16193,"[{'name': 'Aditya Joshi'}, {'name': 'Pranav Go...",22,1610.07091v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",10,Past work in computational sarcasm deals prima...,"[{'term': 'cs.CL', 'scheme': 'http://arxiv.org...",Automatic Identification of Sarcasm Target: An...,2016
5956,[{'name': 'Stefan Hummelsheim'}],10,1401.2304v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,The least absolute shrinkage and selection ope...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Lasso and equivalent quadratic penalized models,2014
21455,"[{'name': 'Ruggiero Lovreglio'}, {'name': 'Vic...",26,1802.09119v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Enhancing evacuee safety is a key factor in re...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Prototyping Virtual Reality Serious Games for ...,2018
11842,"[{'name': 'Shariq Iqbal'}, {'name': 'John Pear...",23,1702.07319v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Despite increasing attention paid to the need ...,"[{'term': 'q-bio.NC', 'scheme': 'http://arxiv....",A Goal-Based Movement Model for Continuous Mul...,2017


In [ ]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [ ]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

for i in range(len(lines)):
    lines[i] = ' '.join(tokenizer.tokenize(lines[i])).lower()

In [ ]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [ ]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens:
# - `UNK` represents absent tokens,
# - `EOS` is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases:
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    for line in tqdm(lines):
        tokens = line.split() + [EOS]

        for i in range(len(tokens)):
            prefix = []

            for j in range(n - 1):
                pos = i - (n - 1) + j
                if pos < 0:
                    prefix.append(UNK)
                else:
                    prefix.append(tokens[pos])
            prefix = tuple(prefix)
            # print(f'Sent: {line} - prefix: {prefix}')

            next_token = tokens[i]

            counts[prefix][next_token] += 1

    return counts


In [ ]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|██████████| 100/100 [00:00<00:00, 9971.48it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [ ]:
class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts

        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n

        self.counts = count_ngrams(lines, self.n)

        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)

        # populate self.probs with actual probabilities
        for prefix in self.counts:
            total = sum(self.counts[prefix].values())
            for word in self.counts[prefix]:
                self.probs[prefix][word] =  self.counts[prefix][word] / total



    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [ ]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|██████████| 100/100 [00:00<00:00, 10452.31it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [ ]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████| 41000/41000 [00:25<00:00, 1580.41it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [ ]:
possible_tokens = lm.get_possible_next_tokens('a machine')

tokens, probs = zip(*possible_tokens.items())
print(f'Tokens: {tokens} - probs: {probs}')
scaled_probs = np.array(probs) ** (1 / 0.5)
print(f'До: {scaled_probs}')
scaled_probs /= scaled_probs.sum()
print(f'После: {scaled_probs}')
random.choices(tokens, weights=scaled_probs, k=1)[0]

Tokens: ("'", 'learning', 'reading', 'that', 'induction', '?', '.', 'to', '-', 'for', 'on', 'translation', ',', 'learner', 'behave', 'and', 'with', 'olfaction', 'models', 'during', 'readable', 'learns', 'translator', 'learned', 'which', 'auxiliary', 'comprehension', 'language', 'processable', 'smart', ':', 'recognition', 'printed', 'vision', 'must', 'capable', ')', 'lies', 'interpretable', 'scheduling', 'across', 'using', 'makes', 'demonstrates', 'or', 'come', 'newly') - probs: (0.0028653295128939827, 0.6045845272206304, 0.014326647564469915, 0.025787965616045846, 0.0028653295128939827, 0.0028653295128939827, 0.017191977077363897, 0.02005730659025788, 0.08022922636103152, 0.008595988538681949, 0.0028653295128939827, 0.06876790830945559, 0.014326647564469915, 0.0057306590257879654, 0.0028653295128939827, 0.0057306590257879654, 0.011461318051575931, 0.0028653295128939827, 0.0028653295128939827, 0.0028653295128939827, 0.008595988538681949, 0.0028653295128939827, 0.0028653295128939827, 0.0

'learning'

In [ ]:
import random
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    possible_tokens = lm.get_possible_next_tokens(prefix)

    if temperature == 0:
        return max(possible_tokens.items(), key=lambda x: x[1])[0]

    tokens, probs = zip(*possible_tokens.items())
    scaled_probs = np.array(probs) ** (1 / temperature)
    scaled_probs /= scaled_probs.sum()
    return random.choices(tokens, weights=scaled_probs, k=1)[0]


In [ ]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [ ]:
prefix = 'machine' # <- your ideas :)

for i in range(50):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

machine learning model is a new network architecture for accelerating algorithmic training and the network configuration can help us answer the question of the afm method , which does not utilize the block structure which best explains the data from wikidata ; the rise of data grows rapidly , irregularly structured


In [ ]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the topological properties of the algorithm . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [ ]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above

    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)

    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    total_logprob = 0.0
    total_tokens = 0
    n = lm.n

    for line in lines:
        tokens = line.split()
        if not tokens:
            continue

        first_token = tokens[0]
        prob = lm.get_next_token_prob('', first_token)
        logprob = np.log(max(prob, np.exp(min_logprob)))
        total_logprob += logprob
        total_tokens += 1

        for i in range(1, len(tokens)):
            next_token = tokens[i]
            prefix = " ".join(tokens[max(0, i - lm.n + 1):i])
            prob = lm.get_next_token_prob(prefix, next_token)
            logprob = np.log(max(prob, np.exp(min_logprob)))
            total_logprob += logprob
            total_tokens += 1

        eos_prefix = ' '.join(tokens[-(n-1):]) if n > 1 else ''
        eos_prob = lm.get_next_token_prob(eos_prefix, EOS)
        logprob = np.log(max(eos_prob, np.exp(min_logprob)))
        total_logprob += logprob
        total_tokens += 1

    total = total_logprob / total_tokens
    return np.exp(-total)

In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("\nPerplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be non-negative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

100%|██████████| 100/100 [00:00<00:00, 11479.61it/s]


Perplexities: ppx1=318.213 ppx3=1.520 ppx10=1.184


Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


100%|██████████| 30750/30750 [00:03<00:00, 9985.01it/s] 


N = 1, Perplexity = 1832.23136


100%|██████████| 30750/30750 [00:06<00:00, 4629.63it/s]


N = 2, Perplexity = 85653987.28774


100%|██████████| 30750/30750 [00:13<00:00, 2319.54it/s]


N = 3, Perplexity = 61999196259043346743296.00000


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|██████████| 100/100 [00:00<00:00, 21951.66it/s]


In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

100%|██████████| 30750/30750 [00:04<00:00, 7235.99it/s] 


N = 1, Perplexity = 977.67559


100%|██████████| 30750/30750 [00:07<00:00, 4076.49it/s]


N = 2, Perplexity = 470.48021


100%|██████████| 30750/30750 [00:22<00:00, 1397.48it/s]


N = 3, Perplexity = 3679.44765


In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
from collections import defaultdict, Counter
from itertools import chain
from tqdm import tqdm

UNK, EOS = "_UNK_", "_EOS_"

class KneserNeyLanguageModel:
    """ Kneser-Ney language model with interpolation smoothing """
    def __init__(self, lines, n, delta=0.75):
        """
        :param lines: an iterable of strings with space-separated tokens
        :param n: n-gram order
        :param delta: discount parameter (typically between 0 and 1)
        """
        self.n = n
        self.delta = delta

        self.counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in self.counts.values() for token in token_counts)

        self.lower_counts = []
        for order in range(1, n):
            self.lower_counts.append(count_ngrams(lines, order))

        self.continuation_counts = defaultdict(Counter)
        self.context_counts = defaultdict(int)

        for prefix, words in self.counts.items():
            for word in words:
                self.continuation_counts[prefix[1:]][word] += 1

        self.word_context_counts = Counter()
        for prefix, words in self.counts.items():
            for word in words:
                self.word_context_counts[word] += 1

        self.total_contexts = sum(len(words) for words in self.counts.values())

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : its probability} for all tokens with positive probabilities
        """
        prefix_tokens = prefix.split()
        prefix_tokens = prefix_tokens[max(0, len(prefix_tokens) - self.n + 1):]
        prefix_tokens = [UNK] * (self.n - 1 - len(prefix_tokens)) + prefix_tokens
        prefix_tuple = tuple(prefix_tokens)

        probs = self._compute_kneser_ney_prob(prefix_tuple)
        return dict(probs)

    def _compute_kneser_ney_prob(self, prefix):
        """
        Compute Kneser-Ney probability for the given prefix
        """
        if len(prefix) == 0:
            total = sum(self.word_context_counts.values())
            probs = Counter()
            for word, count in self.word_context_counts.items():
                probs[word] = count / total
            return probs

        current_counts = self.counts.get(prefix, Counter())
        total_current = sum(current_counts.values())

        if total_current == 0:
            return self._compute_kneser_ney_prob(prefix[1:])

        probs = Counter()
        lambda_prefix = (self.delta * len(current_counts)) / total_current

        for word, count in current_counts.items():
            discounted = max(count - self.delta, 0) / total_current
            probs[word] = discounted

        lower_probs = self._compute_kneser_ney_prob(prefix[1:])
        for word, prob in lower_probs.items():
            probs[word] += lambda_prefix * prob

        return probs

    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

100%|██████████| 100/100 [00:00<00:00, 24797.82it/s]


In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

100%|██████████| 30750/30750 [00:04<00:00, 7308.25it/s]
